In [4]:
import time,os,json
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torch.utils.data import Dataset

In [8]:
os.getcwd()

'c:\\Users\\34644\\Desktop\\Git hub\\dlnn-project_ia-group_1-1\\OUR_utils'

In [2]:


class ConTextDataset(Dataset):
    def __init__(self,dir_images_labels, dir_images, train = True, transform = None):
        self.dir_images        = dir_images
        self.dir_images_labels = dir_images_labels
        self.transform = transform

        if train:
            path = os.path.join(dir_images_labels,'train.txt')
            path_ocr = os.path.join(dir_images_labels, 'ocr_train.txt')
        else:
            path = os.path.join(dir_images_labels,'test.txt')
            path_ocr = os.path.join(dir_images_labels, 'ocr_test.txt')
        
        with open(path, 'r') as file, open(path_ocr, 'r') as ocr_File:
            self.samples = [tuple(line.split(), line_ocr) for line, line_ocr in zip(file, ocr_File)]

    def __len__(self):
        return (len(self.samples))

    def __getitem__(self, idx):
        img_name = os.path.join(self.dir_images, self.samples[idx][0]+'.jpg')
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.samples[idx][2] != '0':
            #Implement text embedding
            pass
        target = torch.tensor(int(self.samples[idx][1]))

        return image, target

ModuleNotFoundError: No module named 'fasttext'